In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel, DataCollatorWithPadding
from transformers import Trainer, TrainingArguments
import pandas as pd
import wandb

In [183]:
train_data = pd.read_csv('si630w22-hw3-train.csv')
dev_data = pd.read_csv('si630w22-hw3-dev.csv')
# test_data = pd.read_csv('data\si630w22-hw3-test.public.csv')
q_and_a_data = pd.read_csv('si630w22-hw3-data.csv')

In [185]:
train_data.rename(columns={'id':'question_id'}, inplace = True)
dev_data.rename(columns={'id':'question_id'}, inplace = True)
train_data.rename(columns={'rating':'label'}, inplace = True)
dev_data.rename(columns={'rating':'label'}, inplace = True)
# test_data.rename(columns={'id':'question_id'}, inplace = True)
q_and_a_data['combined'] = q_and_a_data.question_text + '[SEP]' + q_and_a_data.reply_text

merged_train_data = pd.merge(train_data,q_and_a_data, on='question_id')
merged_train_data.dropna(subset=['label'], inplace = True)
merged_train_data.to_csv('merged_train_data.csv', index=False)

merged_dev_data = pd.merge(dev_data, q_and_a_data, on='question_id')
merged_dev_data.dropna(subset=['label'], inplace = True)
merged_dev_data.to_csv('merged_dev_data.csv', index=False)

# merged_test_data = pd.merge(q_and_a_data, test_data, on='question_id')
# merged_test_data.to_csv('data\merged_test_data.csv', index=False)

In [186]:
data_files = {"train": "merged_train_data.csv",
              "dev": "merged_dev_data.csv",}
#               "test": "data\merged_test_data.csv"}
dataset = load_dataset("csv", data_files=data_files)
dataset

Using custom data configuration default-c0f9a678509c7556


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /home/seffendi/.cache/huggingface/datasets/csv/default-c0f9a678509c7556/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['question_id', 'annotator_id', 'label', 'group', 'question_text', 'reply_id', 'reply_text', 'rlen', 'combined'],
        num_rows: 17842
    })
    dev: Dataset({
        features: ['question_id', 'annotator_id', 'label', 'group', 'question_text', 'reply_id', 'reply_text', 'rlen', 'combined'],
        num_rows: 3809
    })
})

In [187]:
model = AutoModel.from_pretrained("microsoft/MiniLM-L12-H384-uncased")
tokenizer = AutoTokenizer.from_pretrained("microsoft/MiniLM-L12-H384-uncased", padding = True, truncation=True ,max_length =512, return_tensors='pt')

loading configuration file https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/config.json from cache at /home/seffendi/.cache/huggingface/transformers/ceb753d3f27a8c0d09184f35884666cda91b8ae610cd2a54d89793ac7663f1f9.13815020fd994b27db9974c0ce0ec4c47dfac6c8f11bf1a35a0a06d5b165665a
Model config BertConfig {
  "_name_or_path": "microsoft/MiniLM-L12-H384-uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/pytorch_model.b

In [194]:
def tokenize_function(example):
    return tokenizer(example["combined"],padding = 'max_length',truncation=True, max_length =512)


tokenized_dataset = dataset.map(tokenize_function, batched=True, batch_size = len(dataset))
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/8921 [00:00<?, ?ba/s]

  0%|          | 0/1905 [00:00<?, ?ba/s]

In [195]:
# tokenized_dataset
len(tokenized_dataset['train']['input_ids'][7])
# tokenized_dataset['train'][0]

512

In [196]:
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask','label'])
dataloader = torch.utils.data.DataLoader(tokenized_dataset['train'], batch_size=32)
next(iter(dataloader))

{'label': tensor([5., 5., 5., 4., 3., 5., 5., 5., 4., 4., 4., 5., 4., 5., 5., 5., 1., 1.,
         1., 2., 2., 5., 5., 5., 5., 5., 4., 4., 3., 5., 5., 5.]),
 'input_ids': tensor([[ 101, 2054, 1005,  ...,    0,    0,    0],
         [ 101, 2054, 1005,  ...,    0,    0,    0],
         [ 101, 2054, 1005,  ...,    0,    0,    0],
         ...,
         [ 101, 2054, 2024,  ...,    0,    0,    0],
         [ 101, 2054, 2024,  ...,    0,    0,    0],
         [ 101, 2054, 2024,  ...,    0,    0,    0]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])}

In [197]:
training_args = TrainingArguments("test-trainer")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [198]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["dev"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [178]:
# labels

In [199]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertModel.forward` and have been ignored: rlen, question_id, question_text, combined, annotator_id, reply_id, group, reply_text. If rlen, question_id, question_text, combined, annotator_id, reply_id, group, reply_text are not expected by `BertModel.forward`,  you can safely ignore this message.
/home/seffendi/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 17842
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6693
Automatic Weights & Biases logging enabled, to disab

TypeError: forward() got an unexpected keyword argument 'labels'

In [61]:
trainer.train_dataset[1]

{'label': tensor(5.),
 'input_ids': tensor([  101,  2054,  1005,  1055,  2242,  3835,  2017,  2066,  2000,  2079,
          2074,  2000,  2022,  3835,  1029,   102,  2507, 19394,  2015,  1012,
          2009,  1521,  1055,  5186,  3733,  2000,  2079,  1998,  2071,  2689,
          2619,  1521,  1055,  2154,  1012, 10657,  2022,  6176,  2000,  2079,
          2061,  1999,  1037,  2126,  2008,  2987,  1521,  1056,  3499,  1996,
          2060,  2283,  2000,  2228,  2017,  1521,  2128,  7294,  2006,  2068,
          1010,  2295,  1012,  2061,  2009,  1521,  1055,  1037,  5703,  1012,
           102]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1,

In [38]:
tokenizer.decode(trainer.train_dataset[0]["input_ids"])

"[CLS] what's something nice you like to do just to be nice? [SEP] give compliments. it ’ s extremely easy to do and could change someone ’ s day. gotta be careful to do so in a way that doesn ’ t allow the other party to think you ’ re hitting on them, though. so it ’ s a balance. [SEP]"

In [39]:
pd.options.display.max_colwidth = 500
q_and_a_data.combined.head(1)

0    What's something nice you like to do just to be nice?[SEP]Give compliments. It’s extremely easy to do and could change someone’s day. Gotta be careful to do so in a way that doesn’t allow the other party to think you’re hitting on them, though. So it’s a balance.
Name: combined, dtype: object

In [40]:
trainer.train_dataset[0].keys()

dict_keys(['label', 'input_ids', 'token_type_ids', 'attention_mask'])

In [147]:
tokenizer.decode(trainer.train_dataset[0]["label"])

TypeError: Can't convert 5.0 to Sequence

In [110]:
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
labels = torch.tensor([1]).unsqueeze(0)
# model(**inputs)
type(inputs), inputs

(transformers.tokenization_utils_base.BatchEncoding,
 {'input_ids': tensor([[  101,  7592,  1010,  2026,  3899,  2003, 10140,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])})

In [85]:
labels

tensor([[1]])

In [116]:
# x =tokenized_dataset['train'][1]
type(tokenized_dataset['train'][1]), x
for (k, v) in x.items():
    print(k,v)
tuple(x[k] for k in x.keys())

input_ids tensor([  101,  2054,  1005,  1055,  2242,  3835,  2017,  2066,  2000,  2079,
         2074,  2000,  2022,  3835,  1029,   102,  2507, 19394,  2015,  1012,
         2009,  1521,  1055,  5186,  3733,  2000,  2079,  1998,  2071,  2689,
         2619,  1521,  1055,  2154,  1012, 10657,  2022,  6176,  2000,  2079,
         2061,  1999,  1037,  2126,  2008,  2987,  1521,  1056,  3499,  1996,
         2060,  2283,  2000,  2228,  2017,  1521,  2128,  7294,  2006,  2068,
         1010,  2295,  1012,  2061,  2009,  1521,  1055,  1037,  5703,  1012,
          102])
token_type_ids tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
attention_mask tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,


(tensor([  101,  2054,  1005,  1055,  2242,  3835,  2017,  2066,  2000,  2079,
          2074,  2000,  2022,  3835,  1029,   102,  2507, 19394,  2015,  1012,
          2009,  1521,  1055,  5186,  3733,  2000,  2079,  1998,  2071,  2689,
          2619,  1521,  1055,  2154,  1012, 10657,  2022,  6176,  2000,  2079,
          2061,  1999,  1037,  2126,  2008,  2987,  1521,  1056,  3499,  1996,
          2060,  2283,  2000,  2228,  2017,  1521,  2128,  7294,  2006,  2068,
          1010,  2295,  1012,  2061,  2009,  1521,  1055,  1037,  5703,  1012,
           102]),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1

In [150]:
import torch
tokenized_dataset['train']['label'][1]
#  torch.tensor([1]).unsqueeze(0) 

tensor(5.)

In [153]:
a = dataset['train']['combined'][1]
inputs1 = tokenizer(a, return_tensors="pt")
# inputs1
labels = torch.tensor([1] * inputs["input_ids"].size(1)).unsqueeze(0)  # Batch size 1
labels

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [154]:
model(**inputs1)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper__index_select)